# Tokenizing and Translating Markdown with OpenAI

## Introduction

In this notebook, we will implement a pipeline to tokenize a markdown document, split it into chunks at multiple newlines, translate those chunks using OpenAI's LLMs, and reconstruct the full document while retaining original formatting.

Specifically, we will:

- Read in a markdown file 
- Tokenize the text into words, punctuation, etc.
- Count the number of tokens
- Split the tokens into chunks whenever there are multiple successive newlines 
- Translate each chunk into another language using OpenAI's translation LLMs
- Reconstruct the translated chunks into a full document, preserving original formatting like headers, lists, etc.

This allows us to get translations while keeping code blocks, images, tables intact. 

We'll use Python and Jupyter notebooks to implement the pipeline. The notebook will be structured into sections for each step of the process.

To follow along, you'll want a markdown file to process. We'll use a small sample file included with the notebook. You'll need access to OpenAI's API.

Let's get started! First we'll import the modules and setup the notebook. Then we'll define functions for each step - tokenizing, counting tokens, splitting, translating, and reconstructing.

---

### Config

In [1]:
input_language = "english"
output_language = "french" 
format = "markdown" # any special formatting considerations (e.g. .arb file, markdown, json, plain text, or multiple)
splitter = "\n\n" # the split string used to segment the chunks within the text.

### Setup

In [2]:
%pip install --upgrade tiktoken


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import openai, tiktoken

### Import Input File

Import the data file you want translated from `data/input.txt`.

In [4]:
with open("data/input.txt", "r") as f:
    text = f.read()

Length of the text file (for our purposes, we'll be using a small sample text):

In [5]:
len(text)

5667

### Counting the tokens

We'll use OpenAI's `tiktoken` - a fast open-source tokenizer:

> Given a text string (e.g., `"tiktoken is great!"`) and an encoding (e.g., `"cl100k_base"`), a tokenizer can split the text string into a list of tokens (e.g., `["t", "ik", "token", " is", " great", "!"]`).

> Splitting text strings into tokens is useful because GPT models see text in the form of tokens. Knowing how many tokens are in a text string can tell you (a) whether the string is too long for a text model to process and (b) how much an OpenAI API call costs (as usage is priced by token).

In [6]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
encoded_text = encoding.encode(text)
input_token_count = len(encoded_text)


In [7]:
input_token_count

1062

### Split the file into chunks

In [8]:
split_text = text.split(splitter)

The number of chunks:

In [9]:
len(split_text)

16

In [10]:
longest_chunk = max(split_text, key=len)
max_len = len(longest_chunk)

In [11]:
longest_chunk

'*   **Rank Tracking** - Monitoring keyword rankings manually is time consuming. Automated rank trackers refresh data continuously.\n*   **Site Audits** - Crawling sites for issues like broken links and metadata problems is a rote task. Automated site audit tools can find 135+ problems.\n*   **Brand Monitoring** - Tracking brand mentions and monitoring backlinks is tedious without automation. Brand monitoring tools automatically aggregate this data.\n*   **Reporting** - Manual reporting in Excel or Data Studio is inefficient. Automated SEO reporting dashboards provide one-click access to data.\n*   **Image Optimization** - With the rise of visual SERPs, image optimization is critical but laborious. Automated image compressors and upscalers streamline this.\n*   **Site Speed Enhancements** - Page speed improvements like compressing images can be automated to run site-wide.'

In [12]:
max_len

878

In [25]:
system_prompt = f"You are a translation platform. You receive a string in a {format} format and written in {input_language}, and solely return the same string in {output_language} while retaining the {format} formatting. Your translations are accurate, aiming not to deviate from the original structure, content, writing style and tone."

In [26]:
system_prompt

'You are a translation platform. You receive a string in a markdown format and written in english, and solely return the same string in french while retaining the markdown formatting. Your translations are accurate, aiming not to deviate from the original structure, content, writing style and tone.'